<a href="https://colab.research.google.com/github/m-numan1/100-Deep-Learning-Projects/blob/main/mnist_fashion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNIST Fashion Classification Model
**STEPS**
* We will use torchvision Mnist fashion datset
* import all the libraries
* use batches created through dataloader
* Create CNN Neaural Network Model
* train the model
* Evaluate the Model

In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from torchvision.datasets import FashionMNIST

In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((28,28)),
    torchvision.transforms.ToTensor()
])
train_data  = FashionMNIST(root='./data',train=True,download=True,transform=transform)
test_data = FashionMNIST(root='./data',train=False,download=True,transform=transform)


100%|██████████| 26.4M/26.4M [00:01<00:00, 15.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 231kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.29MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.7MB/s]


In [4]:
import matplotlib.pyplot as plt

In [11]:
# Creating a neaural Network

class CNNFashionMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 1 , out_channels = 32, kernel_size = 3 , padding = 1)
    self.conv2 = nn.Conv2d(in_channels = 32 , out_channels = 64, kernel_size = 3 , padding = 1)
    self.conv3 = nn.Conv2d(in_channels = 64 , out_channels = 128, kernel_size = 3 , padding = 1)
    self.pool = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(in_features = 128*3*3, out_features = 128)
    self.fc2 = nn.Linear(in_features = 128, out_features = 10)

  def forward(self, x):

    output = self.pool(F.relu(self.conv1(x)))
    output =self.pool( F.relu(self.conv2(output)))
    output = F.dropout(self.pool(F.relu(self.conv3(output))), p = 0.3)
    output = output.view(output.size(0),-1)
    output = F.relu(self.fc1(output))
    output = self.fc2(output)
    return output


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
train_loader = DataLoader(train_data, batch_size = 100)
test_loader = DataLoader(test_data, batch_size = 100)

In [23]:
# declaring optimizers, loss function, lr, epochs
epochs = 6
learning_rate = 0.001

model = CNNFashionMNIST().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Training loop

for epoch in range(epochs):
  total_loss = 0
  for batch_image , batch_label in train_loader:
    batch_image, batch_label = batch_image.to(device), batch_label.to(device)
    optimizer.zero_grad()
    output = model(batch_image)
    loss = loss_fn(output, batch_label)

    loss.backward()

    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(train_loader)}")


In [18]:
model.eval()

CNNFashionMNIST(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1152, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [20]:
# Evaluating loop

accuracy = 0
total = 0
with torch.no_grad():
  for batch_image, batch_label in test_loader:
    batch_image, batch_label = batch_image.to(device), batch_label.to(device)
    output = model(batch_image)
    _, predicted = torch.max(output.data, 1)
    total += batch_label.size(0)
    accuracy += (predicted == batch_label).sum().item()

print(f"Accuracy: {accuracy/total}")





Accuracy: 0.889


In [22]:
# evaluating on training data

total = 0
accuracy = 0

with torch.no_grad():
  for batch_image, batch_label in train_loader:
    batch_image, batch_label = batch_image.to(device), batch_label.to(device)
    output = model(batch_image)
    _, predicted = torch.max(output.data, 1)
    total += batch_label.size(0)
    accuracy += (predicted == batch_label).sum().item()

print(f"Accuracy: {accuracy/total}")

Accuracy: 0.9009
